<a href="https://colab.research.google.com/github/lankston-consulting/rpms-cutter/blob/main/rpms_cutout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are some libraries that will need to be downloaded if the next cell throws some ModuleNotFound errors. Delete the # of the line who is missing and run the cell.

In [69]:
%%capture
!pip install rasterio
!pip install rioxarray
!pip install ipyleaflet

# **Library Imports and Back End Setup**
Nothing needs to be changed here, this cell just imports all the libraries our code needs to run, and gets Google Drive mounted to this notebook.

You **will** need to approve this notebook to having access to your local Google Drive, it will be a little pop up when this cell gets run!

In [77]:
import os
import datetime
import numpy as np
import xarray as xr
import pandas as pd
import rasterio
import rioxarray
import dask
import tempfile
from rasterio.coords import BoundingBox
from ipyleaflet import *
from google.cloud import storage
from dask.distributed import Client, LocalCluster, Lock, as_completed, fire_and_forget
# from dotenv import load_dotenv
from google.colab import drive
# !gcloud auth login

drive.mount('/content/gdrive')
cluster = LocalCluster(n_workers=8, processes=True)
client = Client(cluster)
path_template = "gs://fuelcast-data/rpms/"
# load_dotenv()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33613 instead
  f"Port {expected} is already in use.\n"


# **Inputs** 

start_year: The beginning year of the range.

end_year: The ending year of the range.

raster_file: The name of the file you dropped into the file space to the left.

**(Optional)** poly_coords: A list of coordinates. This can be typed in manually, but it is highly recommended that you use the drawing tool below.

In [85]:
start_year = 2021
end_year = 2022
file_name = "degradation_bpslut4_bpslut4_wgs84.tif"
poly_coords = {"coords":[]}

# **Optional geometry drawing tool**

In [ ]:
zone_map = Map(center=(38, -97),
                zoom=6,
                basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik))
draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}

def handle_draw(self, action, geo_json):
    poly_coords["coords"] = draw_control.last_draw['geometry']['coordinates'][0]
    print("Done generating coordinates.")
    return poly_coords
    
draw_control.on_draw(handle_draw)
zone_map.add_control(draw_control)

zone_map

# **RPMS Extractor**
You don't need to change anything in here, just let the cell run.

In [ ]:
def pol_to_np(pol):
    """
    Receives list of coordinates: [[x1,y1],[x2,y2],...,[xN,yN]]
    """
    return np.array([list(l) for l in pol])

def pol_to_bounding_box(pol):
    """
    Receives list of coordinates: [[x1,y1],[x2,y2],...,[xN,yN]]
    """
    arr = pol_to_np(pol)
    return BoundingBox(np.min(arr[:,0]),
                       np.min(arr[:,1]),
                       np.max(arr[:,0]),
                       np.max(arr[:,1]))

with rasterio.Env(GOOGLE_APPLICATION_CREDENTIALS="/content/gdrive/Shareddrives/LCLLC/fuelcast-storage-credentials.json"):
  zone_ds = rasterio.open("/content/" + file_name, chunks=(1024, 1024))
  if poly_coords["coords"] != []:
    print("Generating boundary with drawn geometry")
    bounds = pol_to_bounding_box(poly_coords["coords"])
  else:
    print("Generating boundary with imported raster's dimensions")
    bounds = zone_ds.bounds

  profile = zone_ds.profile
  profile.update(blockxsize=1024, blockysize=1024, tiled=True)

  for y in range(start_year, end_year):
      dx = rasterio.open(path_template + str(y) + "/rpms_" + str(y) + ".tif", chunks=(1, 1024, 1024), lock=False)
      op = f"rpms_{str(y)}_mean.tif"
      with rasterio.open('/content/gdrive/My Drive/rpms_'+str(y)+'.tif', 'w', **profile) as dst:
        win = dx.window(bottom=bounds.bottom, right=bounds.right, top=bounds.top, left=bounds.left)
        dat = dx.read(window=win)
        dst.write(dat)